имеется: вертикальная ось опоры, известен азимут разворота опоры  
найти: отклонение оси вдоль и поперек опоры  
значения в градусах, но можно добавить и углы  

In [74]:
from pathlib import Path
import pandas as pd
import numpy as np
from shapely.geometry import Point, MultiPoint, Polygon, LineString
from math import sqrt, acos, cos, sin, degrees, radians

In [2]:
# input data
# для начала возьмем одну опору
t_bot = (35, 45, 10)
t_top = (45, 40, 60)
t_az = (45)


In [9]:
# создаем ось опоры

def create_axe(c_point, az, wide):

    dx = wide*sin(radians(az))
    dy = wide*cos(radians(az))
    
    x1 = c_point[0] + dx
    y1 = c_point[1] + dy
    x2 = c_point[0] - dx
    y2 = c_point[1] - dy

    return LineString([(x1, y1), (x2, y2)])


In [53]:
# определим азимут и расстояние

def azimuth(a, b):
    # by two points we get azimuth
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    dist = sqrt(dx*dx + dy*dy)    # dist a to b
    dx2 = abs(dx)
    beta = degrees(acos(dx2/dist))
    if dx > 0:
        if dy < 0:
            angle = 270 + beta
        else:
            angle = 270 - beta
    else:
        if dy < 0:
            angle = 90 - beta
        else:
            angle = 90 + beta

    return round(angle, 2), dist


In [36]:
# определяем отклонения вдоль и поперек
# dx - поперек, dy - вдоль

def dxdy(t_az, axe, bot_point, top_point):

    # angle and dist to center
    az, dist = azimuth(bot_point[:2], top_point[:2])
        
    new_y = round(Point(top_point[:2]).distance(axe), 2)
    new_x = round(sqrt(abs(dist*dist - new_y*new_y)), 2)
    
    # now we should find + or -
    az -= t_az
    
    if 0 < az < 90:
        new_y *= -1 
    elif 90 < az < 180:
        new_y *= -1
        new_x *= -1
    elif 180 < az < 270:
        new_x *= -1
    elif -90 > az > -180:
        new_x *= -1
    elif -180 > az > -270:
        new_x*= -1
        new_y *= -1
    elif -270 > az > -360:
        new_y *= -1

    return new_x, new_y

In [38]:
# начинаем вычислять

axe = create_axe(t_bot, t_az, 10)   # находим ось

dx, dy = dxdy(t_az, axe, t_bot, t_top)   # находим отклонения

In [39]:
dx, dy

(3.53, -10.61)

In [70]:
# найдем высоту и определим углы отклонений
h = t_top[2] - t_bot[2]
c = sqrt(dx*dx + h*h)
ang_x = degrees(acos(h/c))   # по теореме косинусов прямоугольника

print(ang_x)

4.038381346391643 4.038381346391552


 
### Часть 2 - расчет таблицы
  

In [113]:
# теперь загрузим таблицу и поработаем с ней
workdir = Path(r'D:\work\2022_m\220_man-van\test')

In [82]:
cgt_tab = pd.read_csv(workdir / '220_v2.txt', delimiter='\t', index_col='index', encoding='cp1251')
ang_tab = pd.read_excel(workdir / 'ang_tab.xlsx', usecols='A,H', index_col=0)

In [76]:
cgt_tab

,c_line,id,x,y,z,x_bot,y_bot,z_bot,x_top,y_top,z_top,dist,Unnamed: 13,h
index,,,,,,,,,,,,,,
1,1,ПС_Мангазея,557438.02,7388310.15,34.22,557439.52,7388308.80,19.58,557439.76,7388308.81,34.24,2.02,NaN,14.66
2,1,1,557468.82,7388296.03,45.72,557468.30,7388296.38,18.98,557468.44,7388296.28,45.81,0.63,NaN,26.83
3,1,2,557639.86,7388331.97,50.92,557639.36,7388332.05,19.05,557639.43,7388332.15,51.48,0.51,NaN,32.43
4,1,3,557890.80,7388384.76,50.20,557890.35,7388384.45,18.88,557890.33,7388384.34,50.24,0.55,NaN,31.36
5,1,4,558118.62,7388335.34,43.46,558117.80,7388335.67,17.08,558117.73,7388335.60,43.50,0.88,NaN,26.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,1,195,572291.97,7359966.55,47.17,572291.98,7359966.88,14.06,572291.32,7359966.74,47.27,0.33,NaN,33.21
197,1,ПС_НПС-1,572300.31,7360000.09,29.54,572300.04,7360000.50,16.41,572300.00,7360000.53,32.61,0.49,NaN,16.20
198,2,4,558117.59,7388335.52,43.46,558117.80,7388335.67,17.08,558117.73,7388335.60,43.50,0.26,NaN,26.42


In [107]:
tab_3 = cgt_tab[['c_line', 'id', 'x_bot', 'y_bot', 'z_bot', 'x_top', 'y_top', 'z_top']]

In [108]:
# add column with azimuth
tab_3.insert(8, 'structure azimuth', ang_tab['structure azimuth'])

In [109]:
# add empty columns for dx and dy
#tab_3['dx'] = np.nan
#tab_3['dy'] = np.nan
tab_3.assign(dx=np.nan, dy=np.nan)


,c_line,id,x_bot,y_bot,z_bot,x_top,y_top,z_top,structure azimuth,dx,dy
index,,,,,,,,,,,
1,1,ПС_Мангазея,557439.52,7388308.80,19.58,557439.76,7388308.81,34.24,23.4,NaN,NaN
2,1,1,557468.30,7388296.38,18.98,557468.44,7388296.28,45.81,5.8,NaN,NaN
3,1,2,557639.36,7388332.05,19.05,557639.43,7388332.15,51.48,348.2,NaN,NaN
4,1,3,557890.35,7388384.45,18.88,557890.33,7388384.34,50.24,0.2,NaN,NaN
5,1,4,558117.80,7388335.67,17.08,558117.73,7388335.60,43.50,12.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
196,1,195,572291.98,7359966.88,14.06,572291.32,7359966.74,47.27,314.2,NaN,NaN
197,1,ПС_НПС-1,572300.04,7360000.50,16.41,572300.00,7360000.53,32.61,NaN,NaN,NaN
198,2,4,558117.80,7388335.67,17.08,558117.73,7388335.60,43.50,NaN,NaN,NaN


In [110]:
# теперь в эту таблицу добавляем колонки с тем что делалось на предыдущем этапе
for i in range(len(tab_3)):
    idx = tab_3.iloc[i].name  # find index and work with index
    
    # check if not NaN
    if not pd.isna(tab_3.loc[idx, 'structure azimuth']):
        
    
        # задаем переменные
        t_bot = (tab_3.loc[idx, 'x_bot'], tab_3.loc[idx, 'y_bot'], tab_3.loc[idx, 'z_bot'])
        t_top = (tab_3.loc[idx, 'x_top'], tab_3.loc[idx, 'y_top'], tab_3.loc[idx, 'z_top'])

        t_az = tab_3.loc[idx, 'structure azimuth']    
        if t_az > 180: 
            t_az -= 360    # if > 180: -=360
            tab_3.loc[idx, 'structure azimuth'] = round(t_az, 1)    # replace

        axe = create_axe(t_bot, t_az, 10)   # находим ось
        dx, dy = dxdy(t_az, axe, t_bot, t_top)   # находим отклонения

        tab_3.loc[idx, 'dx'] = dx   # write to the table
        tab_3.loc[idx, 'dy'] = dy


C:\Users\pigse\AppData\Local\Temp\ipykernel_17596\363187059.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab_3.loc[idx, 'dx'] = dx   # write to the table
C:\Users\pigse\AppData\Local\Temp\ipykernel_17596\363187059.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab_3.loc[idx, 'dy'] = dy


In [111]:
tab_3

,c_line,id,x_bot,y_bot,z_bot,x_top,y_top,z_top,structure azimuth,dx,dy
index,,,,,,,,,,,
1,1,ПС_Мангазея,557439.52,7388308.80,19.58,557439.76,7388308.81,34.24,23.4,0.10,-0.22
2,1,1,557468.30,7388296.38,18.98,557468.44,7388296.28,45.81,5.8,-0.08,-0.15
3,1,2,557639.36,7388332.05,19.05,557639.43,7388332.15,51.48,-11.8,0.08,-0.09
4,1,3,557890.35,7388384.45,18.88,557890.33,7388384.34,50.24,0.2,-0.11,0.02
5,1,4,558117.80,7388335.67,17.08,558117.73,7388335.60,43.50,12.1,-0.09,0.05
...,...,...,...,...,...,...,...,...,...,...,...
196,1,195,572291.98,7359966.88,14.06,572291.32,7359966.74,47.27,-45.8,0.38,0.56
197,1,ПС_НПС-1,572300.04,7360000.50,16.41,572300.00,7360000.53,32.61,NaN,NaN,NaN
198,2,4,558117.80,7388335.67,17.08,558117.73,7388335.60,43.50,NaN,NaN,NaN


In [112]:
# save tab
tab_3.to_csv(workdir / '110_tab3.txt', sep='\t', encoding = 'cp1251')